In [313]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
import math

In [4]:
# TRANSITIONS (A,S,S)
# REWARDS     (A,S,S)

In [145]:
# construct possible histories
all_combinations = itertools.product(range(6), repeat=4)
histories = np.asarray([combination for combination in all_combinations if sum(combination) <= 5])
histories[-1], len(histories)

(array([5, 0, 0, 0]), 126)

In [146]:
# construct pendings
all_combinations = itertools.product(range(11), repeat=2)
pendings = np.asarray([combination for combination in all_combinations if sum(combination) <= 20])
pendings[-1], len(pendings)

(array([10, 10]), 121)

In [147]:
pendings[100]

array([9, 1])

^ interpret that as... 9 of 1, 1 of 10

so at most 10 pending at a given time. 2 diff values.

In [148]:
len(histories), len(pendings)

(126, 121)

In [149]:
len(pendings) * len(histories)

15246

In [157]:
def tostr(arr):
    return ','.join(str(x) for x in arr)

In [434]:
states = []
stateInds = {}
ind = 0
for i in tqdm(range(len(pendings))):
    for j in range(len(histories)):
        p, h = pendings[i], histories[j]
        full = np.concatenate((p,h))
        states.append(full)
        strrep = tostr(full)
        stateInds[strrep] = ind
        ind += 1
states = np.asarray(states)
len(stateInds), len(states)

100%|██████████| 121/121 [00:00<00:00, 746.25it/s]


(15246, 15246)

In [360]:
6*15246*15246 # num of elements, need to use sparsity

1394643096

In [833]:
def getReward(state):
    p1, p10 = state[0], state[1]
    return - (p1 + p10*2)

In [834]:
states[12151], getReward(states[12151]) # reward is negative of new state

(array([8, 8, 0, 5, 0, 0]), -24)

In [835]:
# all the values and probabilities for potential new pendings
numProbs = {0:0.5, 1:0.4, 5:0.1}
newPendings = [
    (0,0),
    (1,0),(0,1),
    (5,0),(4,1),(3,2),(2,3),(1,4),(0,5)]
newPendingProbs = [
    (0.9**ones)*(0.1**tens)*math.comb(ones+tens,ones)*numProbs[ones+tens]
    for (ones,tens) in newPendings]
newPendingProbs, sum(newPendingProbs)

([0.5,
  0.36000000000000004,
  0.04000000000000001,
  0.05904900000000001,
  0.032805,
  0.007290000000000002,
  0.0008100000000000004,
  4.500000000000001e-05,
  1.0000000000000004e-06],
 1.0)

In [372]:
### SKIP
curInd = 0
state = states[curInd]
for i, (ones, tens) in enumerate(newPendings):
    newState = np.copy(state)
    newState[0] += ones
    newState[1] += tens
    prob = newPendingProbs[i]
    try:
        newInd = stateInds[tostr(newState)]
    except KeyError: # not a valid new state, leave pendings
        newInd = curInd
    transitions[0][curInd, newInd] = prob
    rewards[0][curInd, newInd] = getReward(newState)
    print(i, state, newState, prob, newInd, getReward(newState))

0 [0 0 0 0 0 0] [0 0 0 0 0 0] 0.5 0 0
1 [0 0 0 0 0 0] [1 0 0 0 0 0] 0.36000000000000004 1386 -1
2 [0 0 0 0 0 0] [0 1 0 0 0 0] 0.04000000000000001 126 -10
3 [0 0 0 0 0 0] [5 0 0 0 0 0] 0.05904900000000001 6930 -5
4 [0 0 0 0 0 0] [4 1 0 0 0 0] 0.032805 5670 -14
5 [0 0 0 0 0 0] [3 2 0 0 0 0] 0.007290000000000002 4410 -23
6 [0 0 0 0 0 0] [2 3 0 0 0 0] 0.0008100000000000004 3150 -32
7 [0 0 0 0 0 0] [1 4 0 0 0 0] 4.500000000000001e-05 1890 -41
8 [0 0 0 0 0 0] [0 5 0 0 0 0] 1.0000000000000004e-06 630 -50


In [836]:
transitions = [lil_matrix((15246,15246), dtype=float) for _ in range(6)]
rewards = [lil_matrix((15246,15246), dtype=float) for _ in range(6)]
transitions[0].size, transitions[0].shape

(0, (15246, 15246))

In [837]:
# action = 0, do nothing, reward from newstate
for curInd, state in tqdm(enumerate(states)):
    for i, (ones, tens) in enumerate(newPendings):
        newState = np.copy(state)
        newState[0] += ones
        newState[1] += tens
        prob = newPendingProbs[i]
        try:
            newInd = stateInds[tostr(newState)]
        except KeyError: # not a valid new state, leave state alone
            newInd = curInd
        transitions[0][curInd, newInd] += prob
        rewards[0][curInd, newInd] = getReward(states[newInd])

15246it [00:03, 4046.36it/s]


In [838]:
# action = a, pop a single wdraw, reward from newstate
for a in range(1,6):
    for curInd, state in tqdm(enumerate(states)):
        interState = np.copy(state)
        reward = 0
        if sum(interState[2:]) + a > 5: # illegal action: violation of constraint
            transitions[a][curInd, curInd] += 1 # stay 
            rewards[a][curInd, curInd] = -10000 # big penalty

        elif sum(interState[:2]) < a:   # illegal action: not enough to process
            transitions[a][curInd, curInd] += 1 # stay
            rewards[a][curInd, curInd] = -10000 # big penalty
        else: # legal action: process
            p1, p10 = interState[0], interState[1]
            remTens = a - p10
            if remTens <= 0:  # only 10s consumed
                p10 -= a
            else:             # use the remainder to process 1s
                p10 = 0
                p1 -= remTens
            interState[0] = p1
            interState[1] = p10

            # shift history
            interState[3:6] = interState[2:5]
            interState[2] = a

            # new pendings
            for i, (ones, tens) in enumerate(newPendings):
                newState = np.copy(interState)
                # add new withdrawals
                newState[0] += ones
                newState[1] += tens
                prob = newPendingProbs[i]
                try:
                    newInd = stateInds[tostr(newState)]
                except KeyError: # not a valid new state, leave state alone
                    newInd = curInd
                transitions[a][curInd, newInd] += prob
                rewards[a][curInd, newInd] = getReward(states[newInd])

15246it [00:02, 6477.08it/s]
15246it [00:01, 11872.81it/s]
15246it [00:00, 21494.34it/s]
15246it [00:00, 33995.34it/s]
15246it [00:00, 45344.50it/s]


In [839]:
transitions = np.asarray([
    csr_matrix(transitions[0]),
    csr_matrix(transitions[1]),
    csr_matrix(transitions[2]),
    csr_matrix(transitions[3]),
    csr_matrix(transitions[4]),
    csr_matrix(transitions[5]),
])
rewards = np.asarray([
    csr_matrix(rewards[0]),
    csr_matrix(rewards[1]),
    csr_matrix(rewards[2]),
    csr_matrix(rewards[3]),
    csr_matrix(rewards[4]),
    csr_matrix(rewards[5]),
])
transitions[1].shape

(15246, 15246)

In [840]:
util.check(transitions, rewards)

In [841]:
vi = mdp.ValueIteration(transitions, rewards, discount=0.9)
vi.run()
vi.iter

97

In [842]:
vi.V[0], vi.policy[0]

(-25.500280857052267, 0)

In [843]:
np.asarray(vi.policy).mean()

0.5900564082382265

In [844]:
policy = np.asarray(vi.policy)
policy.shape

(15246,)

In [845]:
policy.max()

5

In [847]:
# stationary distribution
opt_transitions = lil_matrix((15246,15246), dtype=float)
opt_rewards = lil_matrix((15246,15246), dtype=float)
for i in range(15246):
    opt_transitions[i] = transitions[policy[i]][i]
    opt_rewards[i] = rewards[policy[i]][i]

In [848]:
opt_transitions = csr_matrix(opt_transitions)
opt_rewards = csr_matrix(opt_rewards)

In [849]:
util.check(np.asarray([opt_transitions]), np.asarray([opt_rewards]))

In [850]:
opt_transitions.shape, opt_rewards.shape

((15246, 15246), (15246, 15246))

In [851]:
curr = lil_matrix((1, 15246))
curr[0,0] = 1
iters = 0
while True:
    old = curr.copy()
    curr = curr.dot(opt_transitions)
    diff = np.abs(curr.toarray() - old.toarray())
    if diff.max() < 0.0001:
        print(iters)
        break
    iters += 1

46


In [852]:
opt_rewards.shape, curr.transpose().shape

((15246, 15246), (15246, 1))

In [853]:
vals = opt_rewards.dot(curr.transpose())
vals.shape

(15246, 1)

In [854]:
curr.dot(vals).toarray()

array([[-0.94992563]])

In [855]:
# PRIO-MINSLACK
# stationary distribution
opt_transitions = lil_matrix((15246,15246), dtype=float)
opt_rewards = lil_matrix((15246,15246), dtype=float)
for i, state in enumerate(states):
    slack = 5 - sum(state[2:])
    avail = sum(state[:2])
    policy = 6
    if slack <= avail:
        policy = slack
    else: # avail > slack => can only do avail
        policy = avail
    opt_transitions[i] = transitions[policy][i]
    opt_rewards[i] = rewards[policy][i]

In [856]:
opt_transitions = csr_matrix(opt_transitions)
opt_rewards = csr_matrix(opt_rewards)

In [857]:
util.check(np.asarray([opt_transitions]), np.asarray([opt_rewards]))

In [858]:
opt_transitions.shape, opt_rewards.shape

((15246, 15246), (15246, 15246))

In [859]:
curr = lil_matrix((1, 15246))
curr[0,0] = 1
iters = 0
while True:
    old = curr.copy()
    curr = curr.dot(opt_transitions)
    diff = np.abs(curr.toarray() - old.toarray())
    if diff.max() < 0.0001:
        print(iters)
        break
    iters += 1

415


In [860]:
opt_rewards.shape, curr.transpose().shape

((15246, 15246), (15246, 1))

In [861]:
vals = opt_rewards.dot(curr.transpose())
vals.shape

(15246, 1)

In [862]:
curr.dot(vals).toarray()

array([[-2.83449698]])

In [882]:
# policy
[round(i) for i in np.array([0,1,2,3,4,5]) * 0.85]

[0, 1, 2, 3, 3, 4]

In [891]:
# alpha-MINSLACK
# stationary distribution
opt_transitions = lil_matrix((15246,15246), dtype=float)
opt_rewards = lil_matrix((15246,15246), dtype=float)
for i, state in enumerate(states):
    slack = 5 - sum(state[2:])
    # alpha=0.85
    if slack > 3:
        slack -= 1
    avail = sum(state[:2])
    policy = 6
    if slack <= avail:
        policy = slack
    else: # avail > slack => can only do avail
        policy = avail
    opt_transitions[i] = transitions[policy][i]
    opt_rewards[i] = rewards[policy][i]

In [892]:
opt_transitions = csr_matrix(opt_transitions)
opt_rewards = csr_matrix(opt_rewards)

In [893]:
util.check(np.asarray([opt_transitions]), np.asarray([opt_rewards]))

In [894]:
opt_transitions.shape, opt_rewards.shape

((15246, 15246), (15246, 15246))

In [895]:
curr = lil_matrix((1, 15246))
curr[0,0] = 1
iters = 0
while True:
    old = curr.copy()
    curr = curr.dot(opt_transitions)
    diff = np.abs(curr.toarray() - old.toarray())
    if diff.max() < 0.0001:
        print(iters)
        break
    iters += 1

417


In [896]:
opt_rewards.shape, curr.transpose().shape

((15246, 15246), (15246, 1))

In [897]:
vals = opt_rewards.dot(curr.transpose())
vals.shape

(15246, 1)

In [898]:
curr.dot(vals).toarray()

array([[-2.83564301]])

In [600]:
curr.shape, opt_transitions[0].transpose().shape

((1, 15246), (15246, 1))

In [557]:
curr = lil_matrix((1, 15246))
curr[0,0] = 1
curr

csr_matrix.multipy(curr, )

<1x15246 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in List of Lists format>

In [561]:
curr = np.zeros((1, 2))
curr[0][0] = 1

trans = np.zeros((2,2))
trans[0,0] = 0.3
trans[0,1] = 0.7
trans[1,0] = 0.5
trans[1,1] = 0.5

curr = np.matmul(curr, trans)
curr

array([[0.3, 0.7]])

In [595]:
curr = lil_matrix((1, 2))
curr[0,0] = 1
curr.toarray()

array([[1., 0.]])

In [601]:
curr = lil_matrix((1, 2))
curr[0,0] = 1

trans = lil_matrix((2,2))
trans[0,0] = 0.3
trans[0,1] = 0.7
trans[1,0] = 0.5
trans[1,1] = 0.5

curr = curr.dot(trans)
curr.toarray()

array([[0.3, 0.7]])

In [615]:
curr = curr.dot(trans)
curr.toarray()

array([[0.41666667, 0.58333333]])

In [616]:
opt_transtitions

NameError: name 'opt_transtitions' is not defined

In [547]:
opt_transitions.transpose()[:,0].sum()

1.0

In [553]:
curr = lil_matrix((1, 15246))
curr[0][0] = 1

out = opt_transitions.multiply(opt_transitions)

0.25

In [494]:
curr = np.zeros(15246)
curr[0] = 1 
iters = 0
while True:
    old = np.copy(curr)
    curr = opt_transitions.dot(curr)
    diff = np.abs(curr - old)
    if diff.max() < 0.0001:
        print(iters)
        break
    iters += 1

13


In [502]:
curr.sum()

6.103515625e-05

In [447]:
for i in range(15246):
    pr = transitions[1][i].sum()
    if pr < 0.99 or pr > 1.01:
        print(i)
        break

0


In [448]:
transitions[1][0].sum(), transitions[1][0].indices

(5.0, array([0], dtype=int32))

In [425]:
states[126], states[686]

(array([0, 1, 0, 0, 0, 0]), array([0, 5, 1, 0, 0, 0]))

In [433]:
curInd = 126
state = states[curInd]
interState = np.copy(state)
reward = 0
if sum(interState[2:]) + 1 > 5: # illegal action: violation of constraint
    print("bad")
elif sum(interState[:2]) < 1:   # illegal action: not enough to process
    print("bad")
else: # legal action: process
    print("good")
    p1, p10 = interState[0], interState[1]
    remTens = 1 - p10
    if remTens <= 0:  # only 10s consumed
        p10 -= 1
    else:             # use the remainder to process 1s
        p10 = 0
        p1 -= remTens
    interState[0] = p1
    interState[1] = p10
    print(state)

    interState[3:6] = interState[2:5]
    interState[2] = 1
    print(interState)
    for i, (ones, tens) in enumerate(newPendings):
        newState = np.copy(interState)
        # add new withdrawals
        newState[0] += ones
        newState[1] += tens
        prob = newPendingProbs[i]
        try:
            newInd = stateInds[tostr(newState)]
        except KeyError: # not a valid new state, leave state alone
            newInd = curInd
        print(newState, prob, curInd, newInd)
#     transitions[1][curInd, newInd] += prob
#     rewards[1][curInd, newInd] = getReward(states[newInd])

good
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 1 0 0 0] 0.5 126 56
[1 0 1 0 0 0] 0.36000000000000004 126 1442
[0 1 1 0 0 0] 0.04000000000000001 126 182
[5 0 1 0 0 0] 0.05904900000000001 126 6986
[4 1 1 0 0 0] 0.032805 126 5726
[3 2 1 0 0 0] 0.007290000000000002 126 4466
[2 3 1 0 0 0] 0.0008100000000000004 126 3206
[1 4 1 0 0 0] 4.500000000000001e-05 126 1946
[0 5 1 0 0 0] 1.0000000000000004e-06 126 686


In [401]:
util.check(transitions, rewards)

StochasticError: 'PyMDPToolbox - The transition probability matrix is not stochastic.'

In [3]:
mdp.ValueIteration()

TypeError: __init__() missing 3 required positional arguments: 'transitions', 'reward', and 'discount'